<a href="https://colab.research.google.com/github/nat-smithh/ag-sci/blob/main/microbiome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# microbial community disease risk prediction

In this case study, we'll develop a neural network to predict disease risk from microbial community sequence data.

We have 16S rDNA sequence data from 16,344 samples, roughly half of which are from individuals who have been diagnosed with type 1 diabetes (aka, "cases"), and half of which are from individuals who do not have type 1 diabetes ("controls").

The data are available in a github repository as a comma-separated values (.csv) file. So, we can use the pandas library to download the sequence data and associated disease-state labels to a pandas dataframe:

In [1]:
import pandas
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')
dataframe.head()

,DTA0,DTA1,DTA2,DTA3,DTA4,DTA5,DTA6,DTA7,DTA8,DTA9,...,DTA247,DTA248,DTA249,DTA250,DTA251,DTA252,DTA253,DTA254,DTA255,LBL0
0,1.92,1.80,1.44,1.79,1.68,1.42,1.52,1.58,1.43,1.45,...,-2.02,-2.32,-2.19,-2.25,-2.25,-2.29,-2.19,-2.63,-2.86,1.0
1,1.97,1.98,2.16,2.12,1.78,1.71,1.69,1.60,1.74,1.64,...,-2.05,-1.97,-1.92,-2.12,-1.94,-2.18,-2.45,-2.63,-2.87,0.0
2,2.25,2.11,2.05,1.92,2.08,1.93,1.87,1.57,1.81,1.61,...,-2.02,-1.87,-1.95,-2.09,-1.96,-1.99,-2.01,-2.57,-2.71,1.0
3,2.25,2.07,1.92,1.84,1.83,1.80,1.88,1.48,1.70,1.46,...,-1.94,-2.11,-2.22,-1.98,-2.22,-2.00,-2.10,-2.59,-2.84,0.0
4,2.28,2.27,2.26,2.20,2.01,2.00,1.99,1.92,1.68,1.79,...,-1.69,-1.66,-1.82,-1.88,-1.92,-1.89,-2.07,-2.50,-2.72,0.0


There are 256 "DTA" columns, labelled DTA0, ..., DTA255. Each of these DTA columns represents a particular bacterial "species" found in the samples. The 'relative abundance' of each species (column) in each sample (row) is reported. Relative abundance values have been center log-ratio transformed, which is a common method used to 'normalize' microbial relative abundance data.

In a typical analysis of 16S rDNA sequence data, the 'relative abundance' of each sequence in the sample is given as the *number* of sequence reads matching that sequence in the sample. One *could* divide each sequence count by the total number of counts in that sample, which would produce a typical 'relative abundance' value between 0.0 (not found in the sample) and 1.0 (the *only* sequence found in the sample).

However, it's more common to perform some sort of log-ratio transform of the sequence count data. Log-ratio transforms have a couple of advantages over the 'frequency transform' above. First, putting numbers on a log scale often makes them more 'normally distributed', which typically provides a better fit to the assumptions of most statistical models. Second, the log scale can be 'centered' at zero, with positive and negative values indicating deviations from the 'average' value of zero; this 'centering' often leads to better results from machine-learning and neural-network models.

The center log-ratio transform is simple to calculate and is commonly used for microbial community sequence projects. One simply divides each sequence's count by the *geometric* mean of the total counts over all sequences in the sample, and then takes the logarithm of this 'ratio'.

These data have already been center log-ratio transformed, and you can see that the values typically range between about +2.5 and -2.5.

The final column in the data file, labelled "LBL0" is the 'disease state' indicator (the label we'd like to predict), with 0 indicating a 'control' individual with no type 1 diabetes diagnosis, and 1 indicating a 'case' individual who has been diagnosed with type 1 diabetes.

Our goal is to predict the LBL0 classification, given the microbial sequence information in columns DTA0, ..., DTA255.

First, let's split our data into training and validation subsets, and extract the explanatory variables and labels.

Much of the following code cell should look familiar. Given the pandas dataframe, we first sample 80% of the data for training, and leave the remaining 20% for validation.

Next, we extract the columns starting with "DTA" as the explanatory variables. In this case, we need to 'expand' the data dimension, so we can model these data using a tensorflow sequence model (like a Conv1D or LSTM model).

Finally, we extract the LBL0 entries as our binary class labels.

In [2]:
import numpy as np

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)
print(train_dataframe.shape, valid_dataframe.shape, dataframe.shape)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)
print(train_x.shape, valid_x.shape)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()
print(train_y.shape, valid_y.shape)

(13075, 257) (3269, 257) (16344, 257)
(13075, 256, 1) (3269, 256, 1)
(13075,) (3269,)


We can see that there are 16,344 total samples in our dataframe. We've extracted 13,057 for training and 3,269 for validation.

After expanding the data dimension, we have explanatory variables of shape (256,1), a one-dimensional sequence of 256 bacterial 'species'.

## simple linear model

Now that we have some training and validation data, we just need to build a classifier to predict disease risk from the data.

First, we'll start with a simple linear model implemented using a single Dense neuron with sigmoid output, as this is a binary classification problem.

In previous cases, we were able to send the data directly to the Dense layer. But in this case, because we have 'expanded' the last dimension of the data - in order to fit tensorflow's sequence models - we need to 'collapse' that dimension back down, so it can be properly analyzed by the Dense layer.

It's pretty easy to do this; we just need to use a Flatten layer to 'flatten' the 'expanded' data back down to a simple vector. And, because the Flatten layer is just like any other tensorflow Layer object, we can use it as the *first* layer in our network, provided we set the input_shape option.

The following code cell implements a simple linear classifier for our disease-risk prediction problem.

In [3]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(256,1)))
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257 (1.00 KB)

 Trainable params: 257 (1.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5249 - loss: 0.6929 - val_accuracy: 0.5268 - val_loss: 0.6835
Epoch 2/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5927 - loss: 0.6709 - val_accuracy: 0.5595 - val_loss: 0.6665
Epoch 3/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6439 - loss: 0.6537 - val_accuracy: 0.5941 - val_loss: 0.6506
Epoch 4/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6832 - loss: 0.6380 - val_accuracy: 0.6240 - val_loss: 0.6359
Epoch 5/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7119 - loss: 0.6234 - val_accuracy: 0.6531 - val_loss: 0.6223
Epoch 6/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7355 - loss: 0.6099 - val_accuracy: 0.6773 - val_loss: 0.6096
Epoch 7/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7501 - loss: 0.5973 - val_accuracy: 0.6981 - val_loss: 0.5979
Epoch 8/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7598 - loss: 0.5857 - val_accuracy: 0.

The simple linear model has only 257 trainable parameters (256 input weights, plus the single bias term). But, it does pretty well at classifying our data, given its simplicity.

After 20 epochs of training, my linear model achieved about 80% classification accuracy on the training data, and about 78% classification accuracy on the validation data.

That's a pretty reasonable 'baseline', and it suggests that the *majority* of information required for disease-risk prediction is available in a simple linear model.

Let's see if we can do better using slightly more complex approaches.

## convolution model

The next code cell is an end-to-end example using a simple nonlinear convolution network for disease-risk prediction.

We're using 16 convolution 'filters' of size (3,) in our one-layer network. We've also indicated 'same' padding, so the output of the convolution layer will be the *same* length as the data sequence (256 entries).

We decided to use hyperbolic tangent (tanh) nonlinear activation in this convolution layer. Although this activation is a bit unusual for modern-day convolutions (which more commonly use ReLU activation), it matches the default activation used by recurrent neural networks like LSTMs, so we can more directly compare the results from this convolution network with recurrent network models.

This model will have 4161 trainable parameters, quite a few more than the simple linear model, so we'll train it for a bit longer (50 epochs, in this case).

In [4]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=16,
                                 kernel_size=(3,),
                                 activation=tf.keras.activations.tanh,
                                 padding='same',
                                 input_shape=(256,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 256, 16)        │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161 (16.25 KB)

 Trainable params: 4,161 (16.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5358 - loss: 0.6933 - val_accuracy: 0.7167 - val_loss: 0.6357
Epoch 2/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6658 - loss: 0.6276 - val_accuracy: 0.6546 - val_loss: 0.5961
Epoch 3/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7237 - loss: 0.5635 - val_accuracy: 0.7198 - val_loss: 0.5489
Epoch 4/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7574 - loss: 0.5208 - val_accuracy: 0.7519 - val_loss: 0.5204
Epoch 5/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7781 - loss: 0.4940 - val_accuracy: 0.7629 - val_loss: 0.4998
Epoch 6/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7885 - loss: 0.4745 - val_accuracy: 0.7693 - val_loss: 0.4845
Epoch 7/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7981 - loss: 0.4593 - val_accuracy: 0.7767 - val_loss: 0.4731
Epoch 8/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8062 - loss: 0.4468 - val_accuracy: 0.

Keep track of the time needed to train each epoch, as well as the final accuracy and val_accuracy values, for the associated quiz.

## LSTM model

Finally, we'll try using an LSTM model to analyze the sequence data.

Implement a single-layer LSTM model in the following code cell, using 16 LSTM 'units', and make sure you set return_sequences=True in your model. The rest of the model should use the same Dense decision layer as before.

Remember that LSTM networks can take much longer to train!

In [8]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=2100963)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = np.expand_dims(train_dataframe[dta_ids].to_numpy(), axis=-1)
valid_x = np.expand_dims(valid_dataframe[dta_ids].to_numpy(), axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=16, activation=tf.keras.activations.tanh, input_shape=(256,1), return_sequences=True))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))
model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=50, validation_data=valid_data)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 256, 16)        │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,249 (20.50 KB)

 Trainable params: 5,249 (20.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.5238 - loss: 0.6906 - val_accuracy: 0.7014 - val_loss: 0.6440
Epoch 2/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6651 - loss: 0.6266 - val_accuracy: 0.6932 - val_loss: 0.5615
Epoch 3/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.7556 - loss: 0.5253 - val_accuracy: 0.7388 - val_loss: 0.5130
Epoch 4/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7809 - loss: 0.4780 - val_accuracy: 0.7559 - val_loss: 0.4936
Epoch 5/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.7944 - loss: 0.4497 - val_accuracy: 0.7623 - val_loss: 0.4808
Epoch 6/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7993 - loss: 0.4326 - val_accuracy: 0.7599 - val_loss: 0.4852
Epoch 7/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8033 - loss: 0.4215 - val_accuracy: 0.7550 - val_loss: 0.4957
Epoch 8/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8066 - loss: 0.4139 - val_accu

Make sure you keep track of the training time per epoch, and the final accuracy and loss values, for the associated quiz.

Now that you've tried some 'baseline' models, including a simple linear model and some one-layer convolution and LSTM models, you might want to try 'playing around' with your models, to see if you can improve accuracy without causing too much overfitting.

What happens to the model's accuracy when you increase the number of filters (for convolution networks) or units (for LSTM networks)?

What happens if you add more layers to the networks?

Does training for a longer number of epochs improve accuracy?

When you increase the size of your networks, at what point do you start to see overfitting? Can you 'fix' overfitting with Dropout layers or regularization?

How accurate can you make your model, without too much overfitting?